In [30]:
import numpy as np
import pandas as pd
from datetime import datetime
import re
from collections import Counter
from wordcloud import WordCloud
import warnings
import emoji

In [5]:
def seperate_date_time(x):
    date = pd.to_datetime(x.split(', ')[0])
    time = x.split(', ')[1].split(' - ')[0]
    return date, time

In [6]:
def createDf(file_data, dt):
    patterns = {
        '12 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    
    df = pd.DataFrame({'UserMessage': user_message, 'DateTime': date_time})
    df[['Date','Time']] = df['DateTime'].apply(lambda x: seperate_date_time(x)).to_list()
    users = []
    messages = []
    for message in df['UserMessage']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['User'] = users
    df['Message'] = messages
    df.drop(columns=['UserMessage', 'DateTime'], inplace=True)
    return df

In [7]:
f = open('WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')
df = createDf(f, '12 Hour')

In [8]:
f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '24 Hour'))

In [9]:
f = open('2yearsreport.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '12 Hour'))

In [10]:
f = open('WhatsApp Chat with SAY GT.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '12 Hour'))

In [11]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month_name()
df['MonthNum'] = df['Date'].dt.month
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['WeekNum'] = df['Date'].dt.week
df['Day'] = df['Date'].dt.day
df['DayName'] = df['Date'].dt.day_name()
df['Hour'] = df['Time'].apply(lambda x: x.split(':')[0])
df['Minute'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[0])

#Uncomment while working with 12 hour file format
# df['Meridian'] = df['Time'].apply(lambda x: (x.split(':')[1]).split(' ')[1])  

df.head()

C:\Users\lenovo\AppData\Local\Temp/ipykernel_16212/4283833598.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['WeekNum'] = df['Date'].dt.week


,Date,Time,User,Message,Year,Month,MonthNum,DayOfWeek,WeekNum,Day,DayName,Hour,Minute
0,2022-04-25,8:50 pm,group_notification,Messages and calls are end-to-end encrypted. N...,2022,April,4,0,17,25,Monday,8,50
1,2022-04-25,8:48 pm,group_notification,"Abhsihek Jen Svvv created group ""Say gt""",2022,April,4,0,17,25,Monday,8,48
2,2022-04-25,8:50 pm,group_notification,Abbas added you,2022,April,4,0,17,25,Monday,8,50
3,2022-04-25,8:50 pm,Amit Svvv,ye kis liye?,2022,April,4,0,17,25,Monday,8,50
4,2022-04-25,8:50 pm,group_notification,Abbas added Anil Svvv,2022,April,4,0,17,25,Monday,8,50


# Fetch Stats

In [12]:
def fetch_stats(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  num_messages = df.shape[0]
  diff_days = (df.Date.iloc[len(df)-1] - df.Date.iloc[0]).days
  words = []
  for message in df['Message']:
    words.extend(message.split())

  num_media_messages = df[df['Message'] == '<Media omitted>\n'].shape[0]
  return num_messages, len(words), num_media_messages, diff_days

In [13]:
fetch_stats('Overall', df)

(4201, 21507, 0, 144)

# Removed/Left User

In [14]:
def seperate(x, seperator):
    if seperator == 'removed':
        if x.find(seperator) != -1:
            return x.split(seperator)[1]
        else:
            return x.split('left')[0]
    else:
        if x.find(seperator) != -1:
            return x.split(seperator)[1]
        else:
            return x.split('joined')[0]

In [15]:
seperate('Dhakad Svvv removed +91 94240 96950', 'removed')

' +91 94240 96950'

In [16]:
seperate('You left', 'removed')

'You '

In [17]:
seperate('Dhakad Svvv added +91 98937 63178', 'added')

' +91 98937 63178'

In [18]:
seperate('Hrishikesh Svvv joined using this group', 'added')

'Hrishikesh Svvv '

In [19]:
def removed_left(df):
    df = df[df['User']=='group_notification']
    df = df[df['Message'].str.contains('changed|deleted|encrypted|created') == False]
    removedLeft = df[df['Message'].str.contains('removed|left') ==  True]
    addedJoined = df[df['Message'].str.contains('added|joined') == True]
    removedLeft['Status'] = 0
    addedJoined['Status'] = 1
    removedLeft['User'] = removedLeft.Message.apply(lambda x: seperate(x, 'removed'))
    addedJoined['User'] = addedJoined.Message.apply(lambda x: seperate(x, 'added'))
    new_df = pd.concat([removedLeft, addedJoined])
    new_df.drop(columns=['Date','Time', 'Message', 'Year', 'Month', 'MonthNum', 'DayOfWeek', 'WeekNum', 'Day', 'Hour', 'DayName', 'Minute'], inplace=True, axis=1)
    new_df['User'] = new_df['User'].str.lower()
    new_df['User'] = new_df['User'].str.split(', ')
    new_df = new_df.explode('User')
    new_df['User'] = new_df['User'].str.split(' and ')
    new_df = new_df.explode('User')
    new_df['User'] = new_df['User'].str.strip()
    new_df = new_df.groupby('User')['Status'].count().reset_index()
    new_df['Status'] = new_df.Status.apply(lambda x: x%2 == 0)
    return len(new_df[new_df.Status==True])

In [20]:
removed_left(df)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_16212/962002200.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  removedLeft['Status'] = 0
C:\Users\lenovo\AppData\Local\Temp/ipykernel_16212/962002200.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addedJoined['Status'] = 1
C:\Users\lenovo\AppData\Local\Temp/ipykernel_16212/962002200.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

5

# Chatting From

In [21]:
def chat_from(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['User'] == selected_user]
    unique_years = df['Year'].unique()
    start_year = unique_years[0]
    msg_count = df.groupby(['Date']).count()['Message']
    avg_msg = round(msg_count.mean(),2)
    return start_year, avg_msg

In [22]:
chat_from('Overall', df)

(2022, 34.72)

# Most Talkative

In [23]:
def most_talkative(df):
    df = df[df['User'] != 'group_notification']
    user = df['User'].value_counts()
    username = user.index[0]
    avg_msg = round(user[username]/len(df)*100, 2)
    return username, avg_msg

In [24]:
most_talkative(df)

('Abhsihek Jen Svvv', 13.66)

# Long Winded

In [25]:
def long_winded(df):
    df = df[df['User']!='group_notification']
    new_df = df[df['User']!='<Media omitted>']
    grouped_df = new_df.groupby('User')
    message_length = {}
    for i in new_df['User'].unique():
        print(i+"+++++++")
        _sum = 0
        group = grouped_df.get_group(i)
        for j in group['Message']:
            print(j)
            _sum += len(j)
        print(_sum)
        print("+++++++++")
        print()
        print()
        print()
        print()
        print()
#         message_length[i]=round((_sum/group.shape[0]),2)   
#     name = max(message_length, key=message_length.get)
#     percentage = max(message_length.values())
#     return name, percentage

In [26]:
long_winded(df)

Amit Svvv+++++++
ye kis liye? 
robotics ka assignment 2 bhe 
6 lagana hai bas 
nhii print out ka bola tha bas 
print out  niklwana hh 
okk 
class join krna hai 
link do 
??? 
haa short mai bolu gaa 
minor to ni lge ga maa 
naa* 
This message was deleted 
?? 
pani bhar liya 🤣🤣 
mreko no he ni yaad hai 🙃🙃🙃 
kitne hai 
robotics mai 
9 
10 the shyad 
haa likhu ga hee 
bc 
<Media omitted> 
bas itna he? 
okk 
sailaab ki file hai 
bas iske alava or kch ni hai na? 
ohki.... kitne exp krna hai usme? 
🤣🤣🤣 
haa bhai 10 he kr lee 
file bhe padh lenaa fraud vgraa ky hote hai... 
uske andr he ye sab terms use hai... 
kmean knn to assignment mai.hai 
are vo to.mreko bhe lag rha 
abhi.jaa kr to file complete hui hai meri 🤣🤣🤣 
ab 1-2 ghanta padhu gaa fir gn 😅 
haa mai to bolta hu koi kch mat padh kr aao 
💁‍♂️💁‍♂️ 
haa 
10 experiment likh liye... 
padh rhe ky bhi 
kamadyy king 🤣🤣 
bas sone jaraa 
bhai kch ni padhu gaa kal dhek lege jo hogaa 
🤣🤣 galat fahmii 
hindi mai bol 
chineess 
smjh aai tab to manu

Jha * lga Hora h 
Hmara to parso paper h 
Subh 9 bje 
3 Aug se 
Nhi ki ek b online attend 
<Media omitted> 
Aree kitna b try lr lo...ninja hi lgega sbka 
<Media omitted> 
#include <bits/stdc++.h> using namespace std; #define ll long long int int main() {     int n;     cin>>n;     string a,b;     cin>>a>>b;      set<char> s;   int count=0;     for(int i=0;i<n;i++)     {         if(a[i]!=b[i])         {             if(s.find(b[i])==s.end())             {                 s.insert(b[i]);                 count++;             }          }     }     if(count==n)     {         count=-1;     }     cout<<count<<endl; } 
Online vale ase hi aage nikl jayege... 
Scene doesn't matter 
Code matter 👍 
Nhiiii 
GT done 👍👍 
Hn ab kl aa jate h.. 
To be clear to all 29 Aug to 2 Sept -Betaque -Hexaware -TCS -HSBC Drives would be taking place. 
MST me b GT krna h kya ? 
Sari GT mene hi krvai h 😅 
<Media omitted> 
सत्तासी अस्सी तिरेपन इकतालीस शून्य आठ 
GT 👍 
Abe kl srf 1 ghnte k liye aye...pr parso 5 lecture

Apka dare ye hai ki apko kal college aana hai white dress pehen kr 🤣 
Asvin Nepal 
Ha tera duty ka time ho gya hai na ab Watchmen 
Man of the match award lene gya 
राज़ी नामा 
Retired hai vo 
👻 
Ese pagal mat bol aastha ko 🤣 
Tag krke bol rha 🤣🤣 
Itni izzat hume bhi de diya kr 
Bhai me bhaiya ashvin bhaiya 
Tu ldki h?!? 
Mehndi lagwane jara  Ladiz 
College? 
Terko bulaya bhi nhi 
Kal aana hai 
Simp 
Bong bana liya hoga ab tak kisi ne uss bottle ka 
Ha 
Badhai nhi all the best bolo Interview me elimination jada hota hai 
Thanks 
Thank you 😅 
Thank you 
Thank you 
@919893763178  🎊 
dhanyawad 
kitna time bacha h 
11227
+++++++++





Gopala Svvv+++++++
Jain power hadappa chyata h 
तखता पलट 
9424096950 को remove kr de mera dusra no.h 
File me to shyad print outs h dena h 🤔 
Jinke 10 ho vo ??? Bhe 3 
Yes 
Mtlb b2 wale 2nd shift me aa skte h 😊😊😊 
Unke robotics ka syllabus alag h  Unko computer architecture mix h robotics ke sath 
Kya kya pouch rhe h 
Kiska 
form bhr diya kya ? 
Nhi 
Ohk👍 
Exa

This message was deleted 
Soajo tumne to presentation bhi nhi diya 
😂 
Check the Track my application section in tcs next step if "candidate batched" entry is displayed in the activity section may be you got selected for the next hiring process. 
<Media omitted> 
Ambassador ke 
Shift kya hai isme 
<Media omitted> 
Esliye itna ctc hai 
7/10/12 
😂 
LPA 
Apne batch se 47 LPA amazon kon gya? 
2023 
Wednesday 3rd Lab No alternate teacher for Data Science is assigned, Vijay Sir has left the collage. 
Ab installation kon krwayega 
Wait is semester main Data Science khn se agya 
💐 
Now @919174406481 will become scientist. 
I have seen the last episode of doremon. 
https://youtu.be/RtfYpR28dyo 
Notes bhi nhi send kiye 
No 
Major project hai krlo attend 
Koi nhi arha 
kal nhi aarhe koi bhi 
no 
CR hai vo hod nhi 
Placed now may be 
1 year 
@917772018946 bhi jayega 🙂👍 
27 LPA 
Mars jayega 
This message was deleted 
Don't - Discussion of advance topics. Hr main bhi Tr hoskta hai Do's - Ninja main 

@919171298632 @919713220228 @919755203880 @917582932299 @917440497726  बधाई हो मित्रो 
Kisi k paas big data Hadoop ka mst vala paper h kya apna  Ho to bhejdo.. 
Dhanyavad 
677
+++++++++





Ali Asgar SVVV+++++++
😂 
First three aa jao cloud ka viva start ho gaya hai 
😂😂 
<Media omitted> 
@919755883642 @918435906042 @919174406481 @916263708263 
@919174406481 
This message was deleted 
Congrates bro @919713220228 
Congratulations 
Congratulations 🎉 
229
+++++++++





Achut Svvv+++++++
Leave 14 fab 
👍 
Thanks bro 
Thanks @918435906042 
Thanks bro 
Thanks bhai 
Thanks buddy 
Thanks bro 
Thanks bhai 
106
+++++++++





+91 6239 413 783+++++++
Kisne diye 
Me ujjain aa gya 
MAm ko bol dena me kal kar dunga 
Kal 10:30 bta deta bhai 
Koi bhi mat lana phir 
Scilab matlab me dufference 
Clc and clear command 
2 3×3 ke matrix ko multiply kaose karenge 
Linespace 
How to create script file in matlab and scilan and how to execute 
Han 
👍 
Abey padhlo salo 
Ye kisne diye h? 
Ok 
Bhai notes bhej do y

# Professor

# Emoji Lover

# Early Bird

# Night Owl

# Most Shared Links

Note: you may need to restart the kernel to use updated packages.


# Most Shared Emojis

In [31]:
def emoji_helper(selected_user, df):
    if selected_user != 'Overall':
        df = df[df['user'] == selected_user]
    emojis = []
    description = []
    for message in df['Message']:
        all_emoji = emoji.distinct_emoji_list(message)
        emojis.extend([emoji.emojize(is_emoji) for is_emoji in all_emoji])
    emoji_df = pd.DataFrame(Counter(emojis).most_common(len(Counter(emojis))))
    for i in emoji_df[0]:
        description.append(emoji.demojize(i))
    emoji_df['description'] = description
    return emoji_df[:5],emoji_df

In [32]:
emoji_helper('Overall', df)

AttributeError: module 'emoji' has no attribute 'distinct_emoji_list'

# Hourly Timeline

In [570]:
def hourly_timeline(selected_user, df, format):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df['Message'] = [1] * new_df.shape[0]
  if format == '12 Hour':
    new_df['Hour'] = new_df['Hour'].astype(str) + ' ' + new_df['Meridian'].astype(str)
    new_df = new_df.groupby('Hour').sum().reset_index()
  else:
    new_df = new_df.groupby('Hour').sum().reset_index()
  return new_df

In [571]:
hourly_timeline('Overall', df, '24 Hour')

C:\Users\lenovo\AppData\Local\Temp/ipykernel_1848/631085096.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Message'] = [1] * new_df.shape[0]


,Hour,Message,Year,MonthNum,DayOfWeek,WeekNum,Day
0,1,650,1314300,4040,2163,15824,7798
1,10,345,697590,2191,841,8817,5387
2,11,477,964494,3334,1368,13616,8382
3,12,665,1344630,5569,2209,22606,9090
4,2,202,408444,1211,634,4821,3115
5,3,184,372048,1315,452,5357,3022
6,4,144,291168,1024,397,4146,2243
7,5,219,442818,1088,500,4186,2639
8,6,163,329586,842,530,3145,1430
9,7,161,325542,952,404,3832,2690


# Daily Timeline

In [572]:
def daily_timeline(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df = new_df.groupby('Date')['Message'].count().reset_index()
  return new_df

In [573]:
daily_timeline('Overall', df)

,Date,Message
0,2022-01-05,29
1,2022-01-06,172
2,2022-01-07,6
3,2022-01-09,14
4,2022-02-05,33
...,...,...
113,2022-11-09,53
114,2022-12-05,17
115,2022-12-06,29
116,2022-12-08,23


# Weekly Timeline

In [617]:
def weekly_timeline(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df  = df.groupby(['Year','Month','WeekNum'], sort=False)['Message'].count().reset_index()
  week = []
  for i in range(len(new_df)):
    week.append(str(new_df['WeekNum'][i]) + " - " + new_df['Month'][i] + " - " + str(new_df['Year'][i]))
  new_df['Week'] = week
  new_df['Week'] = new_df[['WeekNum', 'Week']].apply(lambda x: "Week 0"+x['Week'] if x['WeekNum']<10 else "Week "+x['Week'], axis=1)
  new_df.sort_values(['WeekNum', 'Month', 'Year'], inplace=True)
  return new_df

In [618]:
weekly_timeline('Overall', df)

,Year,Month,WeekNum,Message,Week
1,2022,January,1,225,Week 01 - January - 2022
2,2022,February,5,42,Week 05 - February - 2022
20,2022,February,6,77,Week 06 - February - 2022
3,2022,March,9,114,Week 09 - March - 2022
21,2022,March,10,112,Week 10 - March - 2022
4,2022,April,14,210,Week 14 - April - 2022
0,2022,April,17,291,Week 17 - April - 2022
5,2022,May,18,65,Week 18 - May - 2022
13,2022,May,19,149,Week 19 - May - 2022
14,2022,May,20,185,Week 20 - May - 2022


In [449]:
weekly_timeline('+91 6239 413 783', df)

,Year,Month,WeekNum,Message,Week
9,2021,January,1,1,Week 2 - January - 2020
0,2020,January,2,2,Week 51 - December - 2020
7,2021,February,5,1,Week 52 - December - 2020
5,2021,March,12,1,Week 17 - May - 2021
6,2021,April,16,4,Week 22 - June - 2021
3,2021,May,17,1,Week 12 - March - 2021
4,2021,June,22,1,Week 16 - April - 2021
13,2021,June,23,1,Week 5 - February - 2021
8,2021,June,24,5,Week 24 - June - 2021
11,2021,September,37,1,Week 1 - January - 2021


# Monthly Timeline

In [632]:
def monthly_timeline(selected_user, df):
    if selected_user != 'Overall':
      df = df[df['User'] == selected_user]
    new_df = df[df['User'] != 'group_notification']
    new_df  = df.groupby(['Year','Month','MonthNum'], sort=False)['Message'].count().reset_index()
    month = []
    for i in range(new_df.shape[0]):
        month.append(str(new_df['MonthNum'][i]) + " - " + new_df['Month'][i] + " - " + str(new_df['Year'][i]))
    new_df['Months'] = month
    new_df['Months'] = new_df[['MonthNum', 'Months']].apply(lambda x: "Month 0"+x['Months'] if x['MonthNum']<10 else "Month "+x['Months'], axis=1)
    new_df.sort_values(['MonthNum', 'Year'], inplace=True)
    return new_df

In [633]:
monthly_timeline('Overall', df)

,Year,Month,MonthNum,Message,Months
1,2022,January,1,225,Month 01 - January - 2022
2,2022,February,2,119,Month 02 - February - 2022
3,2022,March,3,226,Month 03 - March - 2022
0,2022,April,4,501,Month 04 - April - 2022
4,2022,May,5,626,Month 05 - May - 2022
5,2022,June,6,217,Month 06 - June - 2022
6,2022,July,7,302,Month 07 - July - 2022
7,2022,August,8,948,Month 08 - August - 2022
8,2022,September,9,298,Month 09 - September - 2022
9,2022,October,10,220,Month 10 - October - 2022


# Most Busy Day

In [611]:
def most_busy_day(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df = new_df.groupby(['DayName', 'DayOfWeek'], sort=False)['Message'].count().reset_index()
  new_df.sort_values('DayOfWeek', inplace=True)
  new_df['Days'] = new_df['DayOfWeek'].astype(str) + " - " + new_df['DayName']
  return new_df

In [612]:
most_busy_day('Overall', df)

,DayName,DayOfWeek,Message,Days
0,Monday,0,655,0 - Monday
1,Tuesday,1,500,1 - Tuesday
2,Wednesday,2,586,2 - Wednesday
3,Thursday,3,688,3 - Thursday
4,Friday,4,1025,4 - Friday
5,Saturday,5,423,5 - Saturday
6,Sunday,6,252,6 - Sunday


# Most Busy Month

In [626]:
def most_busy_month(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  
  new_df = new_df.groupby(['MonthNum', 'Month'], sort=False)['Message'].count().reset_index()
  new_df.sort_values('MonthNum', inplace=True)
  new_df['Months'] = new_df['MonthNum'].astype(str) + " - " + new_df['Month']
  new_df['Months'] = new_df[['MonthNum', 'Months']].apply(lambda x: "Month 0"+x['Months'] if x['MonthNum']<10 else "Month "+x['Months'], axis=1)
  return new_df

In [627]:
most_busy_month('Overall', df)
# temp['Months']

,MonthNum,Month,Message,Months
1,1,January,221,Month 01 - January
2,2,February,119,Month 02 - February
3,3,March,223,Month 03 - March
0,4,April,482,Month 04 - April
4,5,May,614,Month 05 - May
5,6,June,216,Month 06 - June
6,7,July,299,Month 07 - July
7,8,August,924,Month 08 - August
8,9,September,297,Month 09 - September
9,10,October,218,Month 10 - October


# User Chat Percentage

In [132]:
def user_chat_percentage(df):
    df = df[df['User'] != 'group_notification']
    user = df['User'].value_counts()
    new_df = pd.DataFrame ({ 'User': user.index, 'Message': user})
    new_df['Percentage'] = new_df['Message'].apply(lambda x: round(x/len(df)*100, 2))
    new_df['User'] = new_df['Percentage'].astype(str) + "% - " + new_df['User']
    new_df.drop('Message', axis=1, inplace=True)
    new_df.reset_index(drop=True,inplace=True)
    return new_df

In [133]:
user_chat_percentage(df)

,User,Percentage
0,15.07% - Clg Ashvin birla,15.07
1,11.49% - Clg Abhishek Jain,11.49
2,9.11% - Clg Kushagra Ahire,9.11
3,8.55% - Clg Amit Pandey,8.55
4,8.27% - Clg Abbas Amjhera,8.27
5,4.13% - Clg Divyansh,4.13
6,4.13% - Clg Charchil,4.13
7,3.92% - Clg Anmol Joshi,3.92
8,3.36% - +91 96309 40258,3.36
9,3.08% - +91 72409 15291,3.08


# Most Common Word

In [168]:
def most_common_words(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df = new_df[~new_df['Message'].str.contains('<Media omitted>')]
  f = open('stop_hinglish.txt')
  stop_words = f.read()

  words = []

  for message in new_df['Message']:
    for word in message.lower().split():
      if word not in stop_words:
        words.append(word)
  
  most_common_df = pd.DataFrame(Counter(words).most_common(20), columns=['Message', 'Count'])
  most_common_df.sort_values('Count')
  return most_common_df

In [169]:
most_common_words('Overall', df)

,Message,Count
0,message,69
1,deleted,67
2,tcs,42
3,form,25
4,jain,23
5,mail,20
6,digital,19
7,drive,18
8,option,17
9,apply,17


# Word Cloud

In [274]:
def create_wordcloud(selected_user, df):
  if selected_user != 'Overall':
    df = df[df['User'] == selected_user]
  new_df = df[df['User'] != 'group_notification']
  new_df = new_df[new_df['Message'] != '<Media omitted>\n']
  
  f = open('stop_hinglish.txt')
  stop_words = f.read()

  def remove_stop_words(message):
    y = []
    for word in message.lower().split():
      if word not in stop_words:
        y.append(word)
    return " ".join(y)

  wc = WordCloud(width=500, height=500, min_font_size=10)
  new_df['Message'] = new_df['Message'].apply(remove_stop_words)
  df_wc = wc.generate(new_df['Message'].str.cat(sep=" "))
  return df_wc

In [275]:
create_wordcloud('Overall', df)

In [406]:
pd.to_datetime('25-04-2022')

Timestamp('2022-04-25 00:00:00')